In [1]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/love_cruise.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round =0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: You have arrived at the port where you are to board a cruise to attend your best friend Alex’s wedding in Svalbard, Norway. Originally you were supposed to take a flight along with everyone else, but due to the limited seat capacity, Alex booked you a cruise instead. The cruise staff checked you in and directed you towards the ship, and urged you to hurry as it is almost time to leave...
actions:
1. Run towards the ship at full speed
2. Take the club car
3. Take your time because why not


I select option 1. TURN: 1 


context: As you dash towards the ship, your pace quick and determined, a passenger named Mandy, in a rush herself, accidentally bumps into you. The collision sends your suit bag flying from your grip, and before you can react, it plummets into the sea below, leaving you in shock as you watch your suit—the one you were supposed to wear at your best friend's wedding—sink into the depths.
actions:
1. Try to retrieve the suit.
2. Apologize to Mandy and continue.
3. Ask Mandy for help.

I select option 1. TURN: 2 


context: You rush to the edge of the dock, eyes scanning the water for any sign of your suit bag. However, the realization quickly sets in that retrieving it from the vast, deep sea is a futile endeavor. The suit is gone, and there's nothing you can do to bring it back. You take a deep breath, trying to calm the panic and frustration building inside you.
actions:
1. Go back and confront Mandy.
2. Head to the ship's lost and found.
3. Continue towards the ship without the suit.

I select option 1. TURN: 3 


context: You turn around, locating Mandy who seems equally flustered by the incident. As you approach her, you notice her apologetic expression. She explains that she was running late to board the ship and didn't see you. Mandy genuinely feels terrible about the accident and offers to compensate you for the lost suit, insisting it's the least she can do.
actions:
1. Accept her offer and exchange contact information.
2. Decline her offer and forgive her.
3. Suggest she helps you find a replacement on the ship.

I select option 1. TURN: 4 


context: You and Mandy exchange contact information, agreeing to sort out the compensation for the lost suit once you're both settled on the ship. Despite the unfortunate start, this moment of kindness from a stranger brings a sense of relief. You thank Mandy for her understanding and generosity, and with a lighter heart, you proceed towards the ship, ready to embark on this unexpected journey.
summary: Jack, in a rush to board the cruise, loses his suit to the sea due to a collision with Mandy. They exchange contact information for compensation.